# Uso da biblioteca SpaCy
### https://spacy.io/
### pip install -U spacy

In [1]:
import pandas as pd

## 1. Importando dados da preparação

In [2]:
PARQUET_MPO_PREPARADOS = '../../Data/Processed/mpo_preparados.parquet'

df_mpo = pd.read_parquet(PARQUET_MPO_PREPARADOS)

## Iniciando: Tokenização, índices e atributos léxicos

In [17]:
# Importe a classe do idioma portugues
from spacy.lang.pt import Portuguese

# Crie um objeto nlp
nlp = Portuguese()

# Processe o texto
doc = nlp("Olá meu nome é Marcelo e tenho 46 anos.")

# Imprima o texto do documento
print(doc.text)

# Imprima alguns atributos léxicos
for token in doc:
    print(f"id: {token.i}\t", f"token: {doc[token.i]}\t", f"pos_i: {token.idx}\t", f"String? {token.is_alpha}\t", f"Número? {token.is_digit}\t", f"Pontuação? {token.is_punct}")

Olá meu nome é Marcelo e tenho 46 anos.
id: 0	 token: Olá	 pos_i: 0	 String? True	 Número? False	 Pontuação? False
id: 1	 token: meu	 pos_i: 4	 String? True	 Número? False	 Pontuação? False
id: 2	 token: nome	 pos_i: 8	 String? True	 Número? False	 Pontuação? False
id: 3	 token: é	 pos_i: 13	 String? True	 Número? False	 Pontuação? False
id: 4	 token: Marcelo	 pos_i: 15	 String? True	 Número? False	 Pontuação? False
id: 5	 token: e	 pos_i: 23	 String? True	 Número? False	 Pontuação? False
id: 6	 token: tenho	 pos_i: 25	 String? True	 Número? False	 Pontuação? False
id: 7	 token: 46	 pos_i: 31	 String? False	 Número? True	 Pontuação? False
id: 8	 token: anos	 pos_i: 34	 String? True	 Número? False	 Pontuação? False
id: 9	 token: .	 pos_i: 38	 String? False	 Número? False	 Pontuação? True


## Importando pequeno modelo treinado em pt
## python -m spacy download pt_core_news_sm

In [18]:
import pt_core_news_sm

nlp = pt_core_news_sm.load()

In [39]:
# Processe o texto
doc = nlp("Olá meu nome é Marcelo e tenho 46 anos.")

# Imprima o texto do documento
print(doc.text)

Olá meu nome é Marcelo e tenho 46 anos.


## Observando previsão de classe gramatical

In [27]:
for token in doc:
    # Imprimir o texto e a classe gramatical prevista (pos_), marcador de dependência (dep_), índice da palavra principal (head) 
    print(token.text, token.pos_, token.dep_, token.head.text)

Olá VERB nsubj Marcelo
meu DET det nome
nome NOUN obj Olá
é AUX cop Marcelo
Marcelo PROPN ROOT Marcelo
e CCONJ cc tenho
tenho VERB conj Marcelo
46 NUM nummod anos
anos NOUN obj tenho
. PUNCT punct Marcelo


In [33]:
import spacy

spacy.explain("DET")


'determiner'

In [32]:
spacy.explain("CCONJ")

'coordinating conjunction'

In [34]:
spacy.explain("nummod")

'numeric modifier'

## Observando as entidades nomeadas identificadas

In [40]:
# Iterar nas entidades previstas
for ent in doc.ents:
    # Imprimir o texto da entidade e seu marcador
    print(ent.text, ent.label_)

Olá MISC
Marcelo MISC


In [36]:
spacy.explain("MISC")

'Miscellaneous entities, e.g. events, nationalities, products or works of art'

In [41]:
doc = nlp("Olá meu nome é Marcelo, moro no Rio de Janeiro e trabalho na Microsoft.")

In [42]:
# Iterar nas entidades previstas
for ent in doc.ents:
    # Imprimir o texto da entidade e seu marcador
    print(ent.text, ent.label_)

Olá MISC
Marcelo MISC
Rio de Janeiro LOC
Microsoft ORG


In [43]:
spacy.explain("LOC")

'Non-GPE locations, mountain ranges, bodies of water'

In [44]:
spacy.explain("ORG")

'Companies, agencies, institutions, etc.'

## Usando comparadores. Além de textos podem comparar atributos léxicos, lematização e outros

In [62]:
# Processe o texto
doc = nlp("Olá meu nome é Marcelo, moro no Rio de Janeiro e trabalho na Apple com iPhone X. Já trabalhei com iPhone 6 e nunca irei trabalhar com o iPhone 13.")

# Imprima o texto do documento
print(doc.text)

Olá meu nome é Marcelo, moro no Rio de Janeiro e trabalho na Apple com iPhone X. Já trabalhei com iPhone 6 e nunca irei trabalhar com o iPhone 13.


In [58]:
# Iterar nas entidades previstas
for ent in doc.ents:
    # Imprimir o texto da entidade e seu marcador
    print(ent.text, ent.label_)

Olá MISC
Marcelo MISC
Rio de Janeiro LOC
Apple ORG


In [111]:
# Importar o Comparador (Matcher)
from spacy.matcher import Matcher

# Inicializar o comparador com o vocabulário. Obrigatório
matcher = Matcher(nlp.vocab)

# Adicionar a expressão ao comparador
pattern = [[{"TEXT": "iPhone"},{"IS_DIGIT":True}]
          ,[{"LOWER": "iphone"},{"LOWER":"x."}]
          ,[{"LEMMA": "trabalhar"}]]


matcher.add("IPHONE_PATTERN", pattern)

# Chamar o matcher no doc
matches = matcher(doc)

# Iterar nas correspondências
for match_id, start, end in matches:
    # Selecionar a partição que houve correspondência
    matched_span = doc[start:end]
    print(matched_span.text)

trabalho
iPhone X.
trabalhei
iPhone 6
trabalhar
iPhone 13
